In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from catboost import CatBoostRegressor,Pool
data1 = pd.read_excel('data.xlsx',sheet_name='soil available Cd dataset')  # 替换为您的数据集文件路径

X = data1[['Soil pH', 'Soil OM (mg/kg)', 'Soil total Cd (mg/kg)']]  # 特征
y1 = data1['Soil available Cd (mg/kg)']  # 第一个目标变量
# 分割数据集
X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.2, random_state=42)
train_pool=Pool(X_train,y1_train)
test_pool=Pool(X_test,y1_test)
total_pool=Pool(X,y1)
model= CatBoostRegressor(learning_rate=0.03,
silent=True,random_seed=42,l2_leaf_reg=6,od_wait=50)
model.fit(train_pool,eval_set=test_pool,use_best_model=True,plot=True)


#use model in test
bestloss=model.get_best_score#best loss
acc=model.score(X_test,y1_test)#准确率
model.plot_tree(tree_idx=0,pool=test_pool)

y_total_pred = model.predict(X)

print(r2_score(y1,y_total_pred))
print(model.get_all_params())#获得所有超参数设置
print(bestloss)
print(acc)
for i,j in zip(X.columns,model.feature_importances_):
    print('{}:{:.2f}%'.format(i,j))

# Make predictions on the training and testing sets
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)
for i,j in zip(X.columns,model.feature_importances_):
    print('{}:{:.2f}%'.format(i,j))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0.9235492315796668
{'nan_mode': 'Min', 'eval_metric': 'RMSE', 'iterations': 1000, 'sampling_frequency': 'PerTree', 'leaf_estimation_method': 'Newton', 'od_pval': 0, 'grow_policy': 'SymmetricTree', 'penalties_coefficient': 1, 'boosting_type': 'Plain', 'model_shrink_mode': 'Constant', 'feature_border_type': 'GreedyLogSum', 'bayesian_matrix_reg': 0.10000000149011612, 'eval_fraction': 0, 'force_unit_auto_pair_weights': False, 'l2_leaf_reg': 6, 'random_strength': 1, 'od_type': 'Iter', 'rsm': 1, 'boost_from_average': True, 'model_size_reg': 0.5, 'pool_metainfo_options': {'tags': {}}, 'subsample': 0.800000011920929, 'use_best_model': True, 'od_wait': 50, 'random_seed': 42, 'depth': 6, 'posterior_sampling': False, 'border_count': 254, 'classes_count': 0, 'auto_class_weights': 'None', 'sparse_features_conflict_fraction': 0, 'leaf_estimation_backtracking': 'AnyImprovement', 'best_model_min_trees': 1, 'model_shrink_rate': 0, 'min_data_in_leaf': 1, 'loss_function': 'RMSE', 'learning_rate': 0.02999

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from catboost import CatBoostRegressor,Pool
data1 = pd.read_excel('data.xlsx',sheet_name='soil available Cd dataset')  # 替换为您的数据集文件路径
data1.head()

,#,Country/region,Soil pH,Soil OM (mg/kg),Soil total Cd (mg/kg),Soil available Cd (mg/kg),References
0,1,China,4.90,60.80,0.510,0.360,14.0
1,2,NaN,5.30,63.30,0.580,0.450,NaN
2,3,NaN,5.22,40.93,0.750,0.480,15.0
3,4,NaN,4.90,15.52,0.727,0.081,16.0
4,5,NaN,5.04,36.60,2.930,0.250,17.0


In [16]:
import pickle
with open ('model.pkl','wb') as f :
    pickle.dump(model,f)
    

In [18]:
with open('model.pkl','rb') as f :
    model2 = pickle.load(f)
    print((X_test[0:2]),model2.predict(X_test[0:2]),
          model.predict(X_test[0:2]))


     Soil pH  Soil OM (mg/kg)  Soil total Cd (mg/kg)
280     6.76             9.73                  0.140
78      7.24            32.85                  0.649 [0.04203946 0.16016158] [0.04203946 0.16016158]


In [20]:
import numpy as np

sample = np.array([6.76, 9.73, 0.140])
print(model.predict(sample.reshape(1, -1)))

[0.04203946]
